In [22]:
from chainer import Chain, optimizers, Variable

from chainer.dataset import iterator
from chainer.iterators import SerialIterator
from chainer.training import Trainer
from chainer.training import StandardUpdater
import numpy as np
import chainer.links as L
from chainer.optimizers import AdaGrad, SGD, MomentumSGD
from chainer.training.extensions import ProgressBar
from chainer.training.extensions import Evaluator, PrintReport, LogReport
import chainer.functions as F
from chainer.optimizer import WeightDecay
from chainer.functions.loss import sigmoid_cross_entropy

import chainer.functions as F
import chainer.links as L

import pandas as pd
import numpy as np

from sklearn import datasets, model_selection
import seaborn as sns
import matplotlib.pyplot as plt
result = pd.read_csv('preprocessed.csv')

# Dataを学習データと検証データに分ける
#X = result.drop('PassengerId', axis=1).drop('Survived', axis=1).drop('TicketNumber', axis=1).drop('TicketLetter', axis=1).drop('Name', axis=1).fillna(0).values.astype(np.float32)
X = result.drop('PassengerId', axis=1).drop('Survived', axis=1).values.astype(np.float32)

y = result['Survived'].values.reshape(len(X), -1).astype(np.int32)

(x_train, x_test, y_train, y_test) = model_selection.train_test_split(
        X,y, test_size=0.3, random_state=0,
        )
n_examples = len(x_train)


In [23]:
class TitanicModel(Chain):
    def __init__(self):
        super(TitanicModel, self).__init__(lin=L.Linear(11, 1))

    def __call__(self, x):
        output = self.lin(x)
        return output


train_data = [(x_train[i,:], y_train[i]) for i in range(n_examples)]
train_iter = SerialIterator(train_data, batch_size = n_examples, repeat=True, shuffle=True)
valid_iter = SerialIterator(train_data, batch_size = 1, repeat=False, shuffle=False)
titanic_model = TitanicModel()
model = L.Classifier(titanic_model, lossfun=sigmoid_cross_entropy.sigmoid_cross_entropy)
model.compute_accuracy = False
opt = MomentumSGD(lr=0.001)
opt.use_cleargrads()
opt.setup(model)
opt.add_hook(WeightDecay(0.0))
updater = StandardUpdater(train_iter, opt, device=-1)
trainer = Trainer(updater, (12000, 'epoch'))
evaluator = Evaluator(valid_iter, model)

trainer.run()


In [24]:
test_pred = F.sigmoid(titanic_model(x_test)).data
test_pred = (test_pred.reshape(len(x_test),) > 0.5).astype(np.int32)
size = len(test_pred)
correct = 0
for i in range(size):
    if test_pred[i] == y_train[i]:
        correct += 1
print(correct/size)

0.5783582089552238


In [ ]:
class MLP(Chain):
    """ 4層のニューラルネットワーク(MLP)
    """
    def __init__(self, n_hid1=100, n_hid2=100, n_out=10):
        # Chainer.Chainクラスを敬称して、Chainクラスの機能を使うためにsuper関数を使う
        super().__init__()

        with self.init_scope():
            self.l1 = L.Linear(None, n_hid1)
            self.l2 = L.Linear(n_hid1, n_hid2)
            self.l3 = L.Linear(n_hid2, n_out)
    def __call__(self, x):
        hid = F.relu(self.l1(x))
        hid2 = F.relu(self.l2(hid))
        return self.l3(hid2)




model = MLP(10, 10, 2)

optimizer = optimizers.SGD()

optimizer.setup(model)

train_data_variable = Variable(train_data.astype(np.float32))
train_label_variable = Variable(train_label.astype(np.int32))

loss_log = []
for epoch in range(200):
    model.cleargrads()

    prod_label = model(train_data_variable)
    loss = F.softmax_cross_entropy(prod_label, train_label_variable)
    loss.backward()
    optimizer.update()
    loss_log.append(loss.data)

plt.plot(loss.data)
